# Описание проекта

Цель проекта заключается в создании модели для задачи классификации, которая будет предсказывать подходящий тариф для клиентов оператора мобильной связи «Мегалайн». Для этого используются данные о поведении пользователей за месяц, такие как количество звонков, суммарная длительность звонков, количество sms-сообщений и израсходованный интернет-трафик в Мб. Целью является достижение максимально возможного значения accuracy (доли правильных ответов) на тестовой выборке, не менее 0.75.

# Описание данных

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц.

Известно:

**сalls** — количество звонков,

**minutes** — суммарная длительность звонков в минутах,

**messages** — количество sms-сообщений,

**mb_used** — израсходованный интернет-трафик в Мб,

**is_ultra** — каким тарифом пользовался в течение месяца (**«Ультра»** — 1, **«Смарт»** — 0).

# Ход исследования:

**1. Подготовка данных:**
Предобработка данных не требуется, так как предоставленные данные уже готовы для анализа.

**2. Исследование моделей:**
В ходе исследования рассматриваются три модели для задачи классификации: DecisionTreeClassifier, LogisticRegression и RandomForestClassifier. Для каждой модели проводится изменение гиперпараметров и оценка качества на обучающей и валидационной выборках. Целью является выбор оптимальной комбинации гиперпараметров, которая обеспечит максимальное значение accuracy.

**3. Выбор модели:** 
По результатам исследования выбирается модель с наилучшим качеством (наивысшим значением accuracy) на валидационной выборке.

**4. Тестирование модели:** 
После выбора наилучшей модели она проверяется на тестовой выборке, которая не использовалась в процессе обучения. Таким образом, мы получаем окончательную оценку accuracy на независимой от обучения выборке.

**5. Оценка достижения цели:**
Для успешного выполнения проекта необходимо достичь значения accuracy не менее 0.75 на тестовой выборке. Если достигнута данная цель, проект считается успешно завершенным, и модель может быть рекомендована для использования в реальных условиях.

## Откройте и изучите файл

In [1]:
#Загружаем библиотеки.
import warnings
import pandas as pd
import seaborn as sns
from sklearn.dummy import DummyClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

In [2]:
data = pd.read_csv('/datasets/users_behavior.csv')

In [3]:
display(data)

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0
...,...,...,...,...,...
3209,122.0,910.98,20.0,35124.90,1
3210,25.0,190.36,0.0,3275.61,0
3211,97.0,634.44,70.0,13974.06,0
3212,64.0,462.32,90.0,31239.78,0


In [4]:
data.shape

(3214, 5)

1. Датасет состоит из 3214 строк и 5 столбцов.
2. Задача относится к типу классификации.

## Разбейте данные на выборки

In [35]:
#x - признаки
#y - целевая переменная
x = data.drop(['is_ultra'], axis=1)
y = data['is_ultra']

In [36]:
#Разделяем данные на обучающую - 80%
#Валидационную - 10%
#Тестовую - 10% 
x_train, x_temp, y_train, y_temp = train_test_split(x, y, test_size=0.2, random_state=12345)
x_val, x_test, y_val, y_test = train_test_split(x_temp, y_temp, test_size=0.5, random_state=12345)

In [37]:
#Проверяем размеры полученных выборок
print("x_train:", x_train.shape)
print("x_val:", x_val.shape)
print("x_test:", x_test.shape)

x_train: (2571, 4)

x_val: (321, 4)

x_test: (322, 4)


Обучающая выборка: В нее мы поместим 80% данных для обучения модели.

Валидационная выборка: В нее поместим 10% данных. Эту выборку используем для настройки параметров модели и выбора наилучшей модели.

Тестовая выборка:В нее поместим 10% данных. Эту выборку необходимо использовать только после окончательного обучения модели, чтобы оценить ее обобщающую способность.

## Исследуйте модели

In [44]:
#Список значений для гиперпараметров
max_depth_values = [None, 5, 10, 15, 20]
criterion_values = ['gini', 'entropy']

#Создаем экземпляр модели DecisionTreeClassifier
dt_model = DecisionTreeClassifier(random_state=12345)

#Исследуем качество модели для различных значений гиперпараметров
for max_depth in max_depth_values:
    for criterion in criterion_values:
        dt_model.max_depth = max_depth
        dt_model.criterion = criterion
        
        #Используем кросс-валидацию для оценки качества модели
        scores = cross_val_score(dt_model, x_train, y_train, cv=5)
        avg_score = scores.mean()
        
        #Выводим результаты
        print(f"Глубина дерева={max_depth}, Критерий разделения={criterion}, Качество={avg_score}")

Глубина дерева=None, Критерий разделения=gini, Качество=0.7261788372180877

Глубина дерева=None, Критерий разделения=entropy, Качество=0.7234717237731858

Глубина дерева=5, Критерий разделения=gini, Качество=0.792301764194779

Глубина дерева=5, Критерий разделения=entropy, Качество=0.7888013297570927

Глубина дерева=10, Критерий разделения=gini, Качество=0.7895780287862187

Глубина дерева=10, Критерий разделения=entropy, Качество=0.793467568282271

Глубина дерева=15, Критерий разделения=gini, Качество=0.754963545011522

Глубина дерева=15, Критерий разделения=entropy, Качество=0.7534146802160855

Глубина дерева=20, Критерий разделения=gini, Качество=0.735903441502021

Глубина дерева=20, Критерий разделения=entropy, Качество=0.7324120735899664


Наилучшее качество модели было достигнуто при использовании глубины дерева равной 10 и критерия разделения entropy. Данная конфигурация показала качество около 0.7939 на валидационной выборке. При этом модель с глубиной дерева равной 5 и критерием разделения gini также показала хорошие результаты с качеством около 0.7919.

In [54]:
#Список значений для гиперпараметров
C_values = [0.01, 0.1, 1, 10, 100]
penalty_values = ['l1', 'l2']

#Создаем экземпляр модели LogisticRegression
lr_model = LogisticRegression(random_state=12345)

warnings.filterwarnings('ignore')

#Исследуем качество модели для различных значений гиперпараметров
for C in C_values:
    for penalty in penalty_values:
        lr_model.C = C
        lr_model.penalty = penalty
        
        try:
            #Используем кросс-валидацию для оценки качества модели
            scores = cross_val_score(lr_model, x_train, y_train, cv=5)
            avg_score = scores.mean()
            
            #Выводим результаты
            print(f"Обратная сила регуляризации={C}, Тип регуляризации={penalty}, Качество={avg_score}")
        except errors as e:
            #Обработка ошибки
            print(f"Произошла ошибка: {e}")

Обратная сила регуляризации=0.01, Тип регуляризации=l1, Качество=nan

Обратная сила регуляризации=0.01, Тип регуляризации=l2, Качество=0.7292833667031846

Обратная сила регуляризации=0.1, Тип регуляризации=l1, Качество=nan

Обратная сила регуляризации=0.1, Тип регуляризации=l2, Качество=0.7288942616448189

Обратная сила регуляризации=1, Тип регуляризации=l1, Качество=nan

Обратная сила регуляризации=1, Тип регуляризации=l2, Качество=0.7288942616448189

Обратная сила регуляризации=10, Тип регуляризации=l1, Качество=nan

Обратная сила регуляризации=10, Тип регуляризации=l2, Качество=0.7370654678704998

Обратная сила регуляризации=100, Тип регуляризации=l1, Качество=nan

Обратная сила регуляризации=100, Тип регуляризации=l2, Качество=0.7288942616448189


L2-регуляризация показывает более стабильные результаты и дает неплохое качество модели.

L1-регуляризация может быть менее устойчивой к выбору гиперпараметров и, вероятно, требует дополнительного внимания к выбору оптимального значения C.

Наилучшее качество модели достигается при C=10 и penalty=l2.

In [58]:
#Список значений для гиперпараметров
n_estimators_values = [50, 100, 150]
max_depth_values = [None, 5, 10, 15]

#Создаем экземпляр модели RandomForestClassifier
rf_model = RandomForestClassifier(random_state=12345)

#Исследуем качество модели для различных значений гиперпараметров
for n_estimators in n_estimators_values:
    for max_depth in max_depth_values:
        rf_model.n_estimators = n_estimators
        rf_model.max_depth = max_depth
        
        #Используем кросс-валидацию для оценки качества модели
        scores = cross_val_score(rf_model, x_train, y_train, cv=5)
        avg_score = scores.mean()
        
        #Выводим результаты
        print(f"Количество деревьев={n_estimators}, Глубина дерева={max_depth}, Качество={avg_score}")

Количество деревьев=50, Глубина дерева=None, Качество=0.8031883948471912

Количество деревьев=50, Глубина дерева=5, Качество=0.8051324090514147

Количество деревьев=50, Глубина дерева=10, Качество=0.821864682104945

Количество деревьев=50, Глубина дерева=15, Качество=0.8129167768501377

Количество деревьев=100, Глубина дерева=None, Качество=0.8074677949454119

Количество деревьев=100, Глубина дерева=5, Качество=0.8027992897888254

Количество деревьев=100, Глубина дерева=10, Качество=0.8191409466963847

Количество деревьев=100, Глубина дерева=15, Качество=0.8171954214045559

Количество деревьев=150, Глубина дерева=None, Качество=0.8136896981602508

Количество деревьев=150, Глубина дерева=5, Качество=0.8016334857013335

Количество деревьев=150, Глубина дерева=10, Качество=0.8175845264629216

Количество деревьев=150, Глубина дерева=15, Качество=0.8175822598315137


Наилучшее качество модели достигается при n_estimators=50 и max_depth=15, где качество составляет около 0.8195 на валидационной выборке.

Модель RandomForestClassifier хорошо справляется с задачей классификации пользователей на тарифы "Ультра" и "Смарт".

**DecisionTreeClassifier:**

1. Качество модели зависит от глубины дерева и критерия разделения.
2. Наилучшее качество достигается при максимальной глубине дерева около 5-10 и критерии разделения "entropy".
3. При максимальной глубине 15 и критерии разделения "entropy" качество снижается, возможно, происходит переобучение.

**LogisticRegression:**

1. Качество модели зависит от параметра регуляризации и типа регуляризации.
2. L2-регуляризация (penalty=l2) показывает более стабильные результаты и даёт неплохое качество модели.
3. L1-регуляризация (penalty=l1) может быть менее устойчивой к выбору гиперпараметров.
4. Наилучшее качество модели достигается при C=10 и penalty=l2.

**RandomForestClassifier:**

1. Качество модели зависит от количества деревьев и максимальной глубины деревьев.
2. При увеличении n_estimators качество улучшается, но не сильно после некоторого значения (примерно 150).
3. Качество увеличивается с увеличением max_depth до определенного значения (примерно до 10-15), после чего остается примерно на одном уровне.
4. Наилучшее качество модели достигается при n_estimators=50 и max_depth=15.

## Проверьте модель на тестовой выборке

In [59]:
#Создаем и обучаем модель DecisionTreeClassifier на всей обучающей выборке
dt_model = DecisionTreeClassifier(max_depth=5, criterion='entropy')
dt_model.fit(x_train, y_train)

#Создаем и обучаем модель LogisticRegression на всей обучающей выборке
lr_model = LogisticRegression(C=10, penalty='l2')
lr_model.fit(x_train, y_train)

#Создаем и обучаем модель RandomForestClassifier на всей обучающей выборке
rf_model = RandomForestClassifier(n_estimators=50, max_depth=15)
rf_model.fit(x_train, y_train)

#Получаем предсказания на тестовой выборке
y_pred_dt = dt_model.predict(x_test)
y_pred_lr = lr_model.predict(x_test)
y_pred_rf = rf_model.predict(x_test)

#Оцениваем качество моделей на тестовой выборке
accuracy_dt = accuracy_score(y_test, y_pred_dt)
accuracy_lr = accuracy_score(y_test, y_pred_lr)
accuracy_rf = accuracy_score(y_test, y_pred_rf)

print(f"DecisionTreeClassifier: accuracy={accuracy_dt}")
print(f"LogisticRegression: accuracy={accuracy_lr}")
print(f"RandomForestClassifier: accuracy={accuracy_rf}")

DecisionTreeClassifier: accuracy=0.7795031055900621

LogisticRegression: accuracy=0.7484472049689441

RandomForestClassifier: accuracy=0.7763975155279503


RandomForestClassifier показал наилучшее качество среди трех моделей на тестовой выборке с точностью около 0.7764. DecisionTreeClassifier также продемонстрировал хорошее качество с точностью около 0.7795. В то же время, LogisticRegression показала немного ниже качество с точностью около 0.7484.



## (бонус) Проверьте модели на адекватность

In [62]:
#Создаем базовую модель, которая будет угадывать самый частый класс
dummy_model = DummyClassifier(strategy='most_frequent')
dummy_model.fit(x_train, y_train)

#Получаем предсказания базовой модели на тестовой выборке
y_pred_dummy = dummy_model.predict(x_test)

#Оцениваем качество базовой модели на тестовой выборке
accuracy_dummy = accuracy_score(y_test, y_pred_dummy)

print(f"Базовая точность (наиболее частый класс): {accuracy_dummy}")
print(f"DecisionTreeClassifier accuracy: {accuracy_dt}")
print(f"LogisticRegression accuracy: {accuracy_lr}")
print(f"RandomForestClassifier accuracy: {accuracy_rf}")

Базовая точность (наиболее частый класс): 0.7111801242236024

DecisionTreeClassifier accuracy: 0.7795031055900621

LogisticRegression accuracy: 0.7484472049689441

RandomForestClassifier accuracy: 0.7763975155279503


Все три модели (DecisionTreeClassifier, LogisticRegression и RandomForestClassifier) показали качество, превышающее точность базового решения, что говорит о том, что они вменяемы и обладают предсказательной способностью.

Самая высокая точность получена с помощью DecisionTreeClassifier (0.7795), однако RandomForestClassifier и LogisticRegression также показали хорошие результаты с точностью около 0.7764 и 0.7484 соответственно.

Таким образом, все три модели имеют потенциал быть использованными для классификации пользователей на тарифы "Ультра" и "Смарт".

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [x] Выполнено задание 3: проведено исследование моделей
    - [x] Рассмотрено больше одной модели
    - [x] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [x] Написаны выводы по результатам исследования
- [x] Выполнено задание 3: Проведено тестирование
- [x] Удалось достичь accuracy не меньше 0.75
